In [195]:
#import dependencis
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix , find
from sklearn.neighbors import NearestNeighbors
from surprise import SVD
import numpy as np
import surprise
import sklearn
from surprise import Reader, Dataset
from datetime import datetime


#### Import Data

In [196]:
movies = pd.read_csv("../raw data/movies.csv")
ratings = pd.read_csv("../raw data/ratings.csv")
links= pd.read_csv("../raw data/links.csv")
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [197]:
ratings.isnull().sum(axis = 0)

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [198]:
print ('number of unique users:',ratings['userId'].nunique())
print ('number of unique:',ratings['movieId'].nunique())

number of unique users: 610
number of unique: 9724


The data set is ready to analyse from source and does not need to be cleaned

In [199]:
ratings = ratings.drop('timestamp',axis=1)

# Split the data

In [200]:
train_data=ratings.iloc[:int(ratings.shape[0]*0.80)]
test_data=ratings.iloc[int(ratings.shape[0]*0.80):]

In [201]:
ratings.shape

(100836, 3)

In [202]:
train_data.shape

(80668, 3)

In [203]:
test_data.shape


(20168, 3)

# SVD MF

https://surprise.readthedocs.io/en/stable/matrix_factorization.html#surprise.prediction_algorithms.matrix_factorization.SVD

Singular Value Decomposition (SVD) is a widely used technique to decompose a matrix into several component matrices, exposing many of the useful and interesting properties of the original matrix.

Matrix factorization is a class of collaborative filtering algorithms used in recommender systems. Matrix factorization algorithms work by decomposing the user-item interaction matrix into the product of two lower dimensionality rectangular matrices.

In [204]:
# Specify how to read the data frame
# https://surprise.readthedocs.io/en/stable/reader.html
reader = Reader(rating_scale=(1,5))

In [205]:
# create the training set from the data frame
train_data_mf = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

# build the train set from train data. 
#It is in dataset format from surprise library
trainset = train_data_mf.build_full_trainset()


In [206]:
# create the test data set from the dataframe
test_data_mf = Dataset.load_from_df(test_data[['userId', 'movieId', 'rating']], reader)

# build the trainset from traindata.., It is of dataset format from surprise library..
testset = test_data_mf.build_full_trainset()

In [207]:
#fitting the model using a matrix factorization-based algorithm
svd = SVD(n_factors=100, biased=True, random_state=15, verbose=True)
svd.fit(trainset)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 5
Processing epoch 6
Processing epoch 7
Processing epoch 8
Processing epoch 9
Processing epoch 10
Processing epoch 11
Processing epoch 12
Processing epoch 13
Processing epoch 14
Processing epoch 15
Processing epoch 16
Processing epoch 17
Processing epoch 18
Processing epoch 19


In [211]:
#getting predictions of train set
train_preds = svd.test(trainset.build_testset())

#for loop to extract prediction and add it to the train_pred_mf df
train_pred_mf = np.array([pred.est for pred in train_preds])


In [212]:
#getting predictions of test set
test_preds = svd.test(testset.build_testset())
test_pred_mf = np.array([pred.est for pred in test_preds])

In [213]:
#calculating the root mean sqaured error of training test 
surprise.accuracy.rmse(train_preds, verbose=True)

RMSE: 0.6455


0.6455039926908807

In [214]:
#calculating the mean absolute error of training test
surprise.accuracy.mae(train_preds, verbose=True)

MAE:  0.5016


0.5016078463508787

In [261]:
optimization_rmse=[]
optimization_mae=[]
optimization_factor= []
optimization_learnrate= []


In [264]:
i=1
for factor in range(200,500,100):
    
    for learnrate in np.arange(0.005, 1, 0.1):
        
        svd = SVD(n_factors=factor, biased=True, random_state=15, lr_all=learnrate ,verbose=False)
        svd.fit(trainset)
        
        #getting predictions of train set
        train_preds = svd.test(trainset.build_testset())
        
        #adding results in list
        optimization_rmse.append(surprise.accuracy.rmse(train_preds, verbose=True))
        optimization_mae.append(surprise.accuracy.mae(train_preds, verbose=True))
        optimization_factor.append(factor)
        optimization_learnrate.append(learnrate)
        print(factor)      
        print('Optimization #',i)
        i=i+1
        print ('----------------------')
    

RMSE: 0.5354
MAE:  0.4176
200
Optimization # 1
----------------------
RMSE: 0.2803
MAE:  0.1840
200
Optimization # 2
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 3
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 4
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 5
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 6
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 7
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 8
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 9
----------------------
RMSE: 1.8073
MAE:  1.4800
200
Optimization # 10
----------------------
RMSE: 0.4659
MAE:  0.3641
300
Optimization # 11
----------------------
RMSE: 0.2790
MAE:  0.1826
300
Optimization # 12
----------------------
RMSE: 1.8073
MAE:  1.4800
300
Optimization # 13
----------------------
RMSE: 1.8073
MAE:  1.4800
300
Optimization # 14
----------------------
RMSE: 1.8073
MA

In [266]:
optimization_grid = pd.DataFrame(
    {'RMSE': optimization_rmse,
     'MAE': optimization_mae,
     'Factors':optimization_factor,
     'Learning Rate' : optimization_learnrate
    })

optimization_grid.head()


,RMSE,MAE,Factors,Learning Rate
0,0.535439,0.417572,200,0.005
1,0.280285,0.184001,200,0.105
2,1.807258,1.480023,200,0.205
3,1.807258,1.480023,200,0.305
4,1.807258,1.480023,200,0.405


In [276]:
minrmse = optimization_grid['RMSE'].min()
minrmse_index = optimization_grid['RMSE'].idxmin()
print("min RMSE:",minrmse)
print("min RMSE index:" ,minrmse_index)

min RMSE: 0.2781744727258595
min RMSE index: 21


In [278]:
minmae = optimization_grid['MAE'].min()
minmae_index = optimization_grid['MAE'].idxmin()
print("min MAE:",minmae)
print("min MAE index:" ,minmae_index)

min MAE: 0.1823900830235546
min MAE index: 21


both min MAE and and RMSE are from the same iteration number will extract number of factors and learning rate

In [280]:
print("Number of factors:" ,optimization_grid['Factors'][21])
print("Learning Rate:", optimization_grid['Learning Rate'][21])

Number of factors: 400
Learning Rate: 0.10500000000000001


will train the model with the optimized parameters now

In [281]:
#fitting the model using a matrix factorization-based algorithm with optimized parameters
svd = SVD(n_factors=400, biased=True, random_state=15, lr_all=0.10500000000000001 ,verbose=False)
svd.fit(trainset)
#getting predictions of train set
train_preds = svd.test(trainset.build_testset())

#for loop to extract prediction and add it to the train_pred_mf df
train_pred_mf = np.array([pred.est for pred in train_preds])

#getting predictions of test set
test_preds = svd.test(testset.build_testset())
test_pred_mf = np.array([pred.est for pred in test_preds])
#calculating the root mean sqaured error of training test 
surprise.accuracy.rmse(train_preds, verbose=True)
#calculating the mean absolute error of training test
surprise.accuracy.mae(train_preds, verbose=True)


RMSE: 0.2782
MAE:  0.1824


0.1823900830235546

The starting RMSE for SVD MF is 0.65, and MAE is 0.50. With optimization we were able to reduce it RMSE to 0.28 and MAE to 0.18. We will next explore the XGBoost algorithm. We will also use the predictions from this training/testing to add as a column in XGBoost as a predictor as well

# XGBoost

XGBoost is an implementation of gradient boosted decision trees designed for speed and performance. XGBoost is a software library that you can download and install on your machine (pip install XGBoost). XGBoost dominates structured or tabular datasets on classification and regression predictive modeling problems.

Gradient boosting is a machine learning technique for regression, classification and other tasks, which produces a prediction model in the form of an ensemble of weak prediction models, typically decision trees. When a decision tree is the weak learner, the resulting algorithm is called gradient boosted trees, which usually outperforms random forest. It builds the model in a stage-wise fashion like other boosting methods do, and it generalizes them by allowing optimization of an arbitrary differentiable loss function. (wikipedia)

https://machinelearningmastery.com/gentle-introduction-xgboost-applied-machine-learning/



## Prepare train data set

In [35]:
# Creating a sparse matrix
# https://machinelearningmastery.com/sparse-matrices-for-machine-learning/

train_sparse_matrix = csr_matrix((train_data.rating.values, (train_data.userId.values,
                                               train_data.movieId.values)))

In [36]:
train_averages = dict()
# get the global average of ratings in our train set.
train_global_average = train_sparse_matrix.sum()/train_sparse_matrix.count_nonzero()
train_averages['global'] = train_global_average
train_averages

{'global': 3.5199769425298757}

In [37]:
# get the user averages in dictionary (key: user_id/movie_id, value: avg rating)
def get_average_ratings(sparse_matrix, of_users):
    
    # average ratings of user/axes
    ax = 1 if of_users else 0 # 1 - User axes,0 - Movie axes
    # ".A1" is for converting Column_Matrix to 1-D numpy array
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Boolean matrix of ratings ( whether a user rated that movie or not)
    is_rated = sparse_matrix!=0
    # no of ratings that each user OR movie..
    no_of_ratings = is_rated.sum(axis=ax).A1
    # max_user and max_movie ids in sparse matrix
    u,m = sparse_matrix.shape
    # create a dictionary of users and their average ratings..
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
    for i in range(u if of_users else m)
    if no_of_ratings[i] !=0}
    #return that dictionary of average ratings
    return average_ratings

In [38]:
#average ratings given by a user for all movies
train_averages['user'] = get_average_ratings(train_sparse_matrix, of_users=True)

In [39]:
#average ratings given for a movie by all users
train_averages['movie'] = get_average_ratings(train_sparse_matrix, of_users=False)

In [45]:
# get users, movies and ratings from our samples train sparse matrix
train_users, train_movies, train_ratings = find(train_sparse_matrix)

### it took about 50 mins for this chunk to run so run when you have time

In [54]:
# it took about 50 mins for this chunk to run
start =datetime.now()
final_data = pd.DataFrame()
count = 0
for (user, movie, rating)  in zip(train_users, train_movies, train_ratings):
            st = datetime.now()
        #     print(user, movie)    
            #--------------------- Ratings of "movie" by similar users of "user" ---------------------
            # compute the similar Users of the "user"        
            user_sim = sklearn.metrics.pairwise.cosine_similarity(train_sparse_matrix[user], train_sparse_matrix).ravel()
            top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar users for this movie
            top_ratings = train_sparse_matrix[top_sim_users, movie].toarray().ravel()
            # we will make it's length "5" by adding movie averages to .
            top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_users_ratings.extend([train_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
        #     print(top_sim_users_ratings, end=" ")    


            #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
            # compute the similar movies of the "movie"        
            movie_sim = sklearn.metrics.pairwise.cosine_similarity(train_sparse_matrix[:,movie].T, train_sparse_matrix.T).ravel()
            top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar movie rated by this user..
            top_ratings = train_sparse_matrix[user, top_sim_movies].toarray().ravel()
            # we will make it's length "5" by adding user averages to.
            top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movies_ratings.extend([train_averages['user'][user]]*(5-len(top_sim_movies_ratings))) 
        #     print(top_sim_movies_ratings, end=" : -- ")

            #-----------------prepare the row to be stores in a file-----------------#
            row = list()
            row.append(user)
            row.append(movie)
            # Now add the other features to this data...
            row.append(train_averages['global']) # first feature
            # next 5 features are similar_users "movie" ratings
            row.extend(top_sim_users_ratings)
            # next 5 features are "user" ratings for similar_movies
            row.extend(top_sim_movies_ratings)
            # Avg_user rating
            row.append(train_averages['user'][user])
            # Avg_movie rating
            row.append(train_averages['movie'][movie])

            # finalley, The actual Rating of this user-movie pair...
            row.append(rating)
            count = count + 1
            final_data = final_data.append([row])
            #print(count)

        
            if (count)%10000 == 0:
                # print(','.join(map(str, row)))
                print("Done for {} rows----- {}".format(count, datetime.now() - start))

Done for 10000 rows----- 0:05:55.452014
Done for 20000 rows----- 0:11:57.418994
Done for 30000 rows----- 0:17:53.886895
Done for 40000 rows----- 0:23:59.708944
Done for 50000 rows----- 0:29:57.049033
Done for 60000 rows----- 0:35:58.676074
Done for 70000 rows----- 0:42:12.192363
Done for 80000 rows----- 0:49:36.175306


In [55]:
final_data.columns=['user', 'movie', 'GAvg', 'sur1', 'sur2', 'sur3', 'sur4', 'sur5',
            'smr1', 'smr2', 'smr3', 'smr4', 'smr5', 'UAvg', 'MAvg', 'rating']


In [56]:
final_data.shape


(80668, 16)

In [57]:

final_data.head()

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating
0,1,1,3.519977,2.0,5.0,4.0,4.0,4.5,3.0,4.0,3.0,5.0,5.0,4.366379,3.954545,4.0
0,5,1,3.519977,4.0,5.0,4.0,4.0,5.0,4.0,3.0,3.0,5.0,5.0,3.636364,3.954545,4.0
0,7,1,3.519977,4.0,4.0,5.0,4.5,4.0,4.5,4.5,5.0,4.0,3.0,3.230263,3.954545,4.5
0,15,1,3.519977,5.0,3.0,4.0,4.0,4.0,3.5,3.0,5.0,3.0,3.0,3.448148,3.954545,2.5
0,17,1,3.519977,4.0,5.0,4.0,4.0,4.5,4.0,4.5,5.0,5.0,5.0,4.209524,3.954545,4.5


In [58]:
final_data['mf_svd']=train_pred_mf


In [59]:
final_data.head()

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd
0,1,1,3.519977,2.0,5.0,4.0,4.0,4.5,3.0,4.0,3.0,5.0,5.0,4.366379,3.954545,4.0,4.306195
0,5,1,3.519977,4.0,5.0,4.0,4.0,5.0,4.0,3.0,3.0,5.0,5.0,3.636364,3.954545,4.0,3.941735
0,7,1,3.519977,4.0,4.0,5.0,4.5,4.0,4.5,4.5,5.0,4.0,3.0,3.230263,3.954545,4.5,4.543556
0,15,1,3.519977,5.0,3.0,4.0,4.0,4.0,3.5,3.0,5.0,3.0,3.0,3.448148,3.954545,2.5,4.677380
0,17,1,3.519977,4.0,5.0,4.0,4.0,4.5,4.0,4.5,5.0,5.0,5.0,4.209524,3.954545,4.5,4.810359


# Preparing test data

In [60]:
# Creating a sparse matrix
test_sparse_matrix = csr_matrix((test_data.rating.values, (test_data.userId.values,
                                               test_data.movieId.values)))

In [61]:
# Global avg of all movies by all users

test_averages = dict()
# get the global average of ratings in our train set.
test_global_average = test_sparse_matrix.sum()/test_sparse_matrix.count_nonzero()
test_averages['global'] = test_global_average
test_averages

{'global': 3.4278808012693376}

In [62]:
# get the user averages in dictionary (key: user_id/movie_id, value: avg rating)

def get_average_ratings(sparse_matrix, of_users):
    
    # average ratings of user/axes
    ax = 1 if of_users else 0 # 1 - User axes,0 - Movie axes

    # ".A1" is for converting Column_Matrix to 1-D numpy array 
    sum_of_ratings = sparse_matrix.sum(axis=ax).A1
    # Boolean matrix of ratings ( whether a user rated that movie or not)
    is_rated = sparse_matrix!=0
    # no of ratings that each user OR movie..
    no_of_ratings = is_rated.sum(axis=ax).A1
    
    # max_user  and max_movie ids in sparse matrix 
    u,m = sparse_matrix.shape
    # creae a dictonary of users and their average ratigns..
    average_ratings = { i : sum_of_ratings[i]/no_of_ratings[i]
                                 for i in range(u if of_users else m) 
                                    if no_of_ratings[i] !=0}

    # return that dictionary of average ratings
    return average_ratings

In [63]:
# Average ratings given by a user

test_averages['user'] = get_average_ratings(test_sparse_matrix, of_users=True)
#print('\nAverage rating of user 10 :',test_averages['user'][10])

# Average ratings given for a movie

test_averages['movie'] =  get_average_ratings(test_sparse_matrix, of_users=False)
#print('\n AVerage rating of movie 15 :',test_averages['movie'][15])

In [65]:

# get users, movies and ratings from our samples train sparse matrix
test_users, test_movies, test_ratings = find(test_sparse_matrix)

### takes about 2 mins to run this chunk

In [67]:
final_test_data = pd.DataFrame()
count = 0
for (user, movie, rating)  in zip(test_users, test_movies, test_ratings):
            st = datetime.now()
        #     print(user, movie)    
            #--------------------- Ratings of "movie" by similar users of "user" ---------------------
            # compute the similar Users of the "user"        
            user_sim = sklearn.metrics.pairwise.cosine_similarity(test_sparse_matrix[user], test_sparse_matrix).ravel()
            top_sim_users = user_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar users for this movie
            top_ratings = test_sparse_matrix[top_sim_users, movie].toarray().ravel()
            # we will make it's length "5" by adding movie averages to .
            top_sim_users_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_users_ratings.extend([test_averages['movie'][movie]]*(5 - len(top_sim_users_ratings)))
        #     print(top_sim_users_ratings, end=" ")    


            #--------------------- Ratings by "user"  to similar movies of "movie" ---------------------
            # compute the similar movies of the "movie"        
            movie_sim = sklearn.metrics.pairwise.cosine_similarity(test_sparse_matrix[:,movie].T, test_sparse_matrix.T).ravel()
            top_sim_movies = movie_sim.argsort()[::-1][1:] # we are ignoring 'The User' from its similar users.
            # get the ratings of most similar movie rated by this user..
            top_ratings = test_sparse_matrix[user, top_sim_movies].toarray().ravel()
            # we will make it's length "5" by adding user averages to.
            top_sim_movies_ratings = list(top_ratings[top_ratings != 0][:5])
            top_sim_movies_ratings.extend([test_averages['user'][user]]*(5-len(top_sim_movies_ratings))) 
        #     print(top_sim_movies_ratings, end=" : -- ")

            #-----------------prepare the row to be stores in a file-----------------#
            row = list()
            row.append(user)
            row.append(movie)
            # Now add the other features to this data...
            row.append(test_averages['global']) # first feature
            # next 5 features are similar_users "movie" ratings
            row.extend(top_sim_users_ratings)
            # next 5 features are "user" ratings for similar_movies
            row.extend(top_sim_movies_ratings)
            # Avg_user rating
            row.append(test_averages['user'][user])
            # Avg_movie rating
            row.append(test_averages['movie'][movie])

            # finalley, The actual Rating of this user-movie pair...
            row.append(rating)
            count = count + 1
            final_test_data = final_test_data.append([row])
            #print(count)

           
        
            if (count)%10000 == 0:
                # print(','.join(map(str, row)))
                print("Done for {} rows----- {}".format(count, datetime.now() - start))

Done for 10000 rows----- 1:41:41.972823
Done for 20000 rows----- 1:46:24.218922


In [68]:

final_test_data.columns=['user', 'movie', 'GAvg', 'sur1', 'sur2', 'sur3', 'sur4', 'sur5',
            'smr1', 'smr2', 'smr3', 'smr4', 'smr5', 'UAvg', 'MAvg', 'rating']

In [69]:
final_test_data.shape


(20168, 16)

In [70]:
test_pred_mf.shape


(20168,)

In [71]:
final_test_data['mf_svd']=test_pred_mf


# Creating XGBoost

In [358]:
def get_error_metrics(y_true, y_pred):
    rmse = np.sqrt(np.mean([ (y_true[i] - y_pred[i])**2 for i in range(len(y_pred)) ]))
    mape = np.mean(np.abs( (y_true - y_pred)/y_true )) * 100
    return rmse, mape

In [359]:
# prepare train data
x_train = final_data.drop(['user', 'movie','rating'], axis=1)
y_train = final_data['rating']

In [360]:
# Prepare Test data
x_test = final_test_data.drop(['user','movie','rating'], axis=1)
y_test = final_test_data['rating']

In [361]:
import xgboost as xgb


In [362]:
# initialize XGBoost model
xgb_model = xgb.XGBRegressor(silent=False, n_jobs=13, random_state=15, n_estimators=100)
# dictionaries for storing train and test results
train_results = dict()
test_results = dict()
    
    
# fit the model
print('Training the model..')
start =datetime.now()
xgb_model.fit(x_train, y_train, eval_metric = 'rmse')
print('Done. Time taken : {}\n'.format(datetime.now()-start))
print('Done \n')

Training the model..
[18:23:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Done. Time taken : 0:00:11.121539

Done 



In [283]:
#available paramaters to tune
xgb_model.get_params()

{'objective': 'reg:squarederror',
 'base_score': 0.5,
 'booster': 'gbtree',
 'colsample_bylevel': 1,
 'colsample_bynode': 1,
 'colsample_bytree': 1,
 'gamma': 0,
 'gpu_id': -1,
 'importance_type': 'gain',
 'interaction_constraints': '',
 'learning_rate': 0.300000012,
 'max_delta_step': 0,
 'max_depth': 6,
 'min_child_weight': 1,
 'missing': nan,
 'monotone_constraints': '()',
 'n_estimators': 100,
 'n_jobs': 13,
 'num_parallel_tree': 1,
 'random_state': 15,
 'reg_alpha': 0,
 'reg_lambda': 1,
 'scale_pos_weight': 1,
 'subsample': 1,
 'tree_method': 'exact',
 'validate_parameters': 1,
 'verbosity': None,
 'silent': False}

In [363]:
# from the trained model, get the predictions
print('Evaluating the model with TRAIN data')
start =datetime.now()
y_train_pred = xgb_model.predict(x_train)
# get the rmse and mape of train data...
rmse_train, mape_train = get_error_metrics(y_train.values, y_train_pred)
    
# store the results in train_results dictionary..
train_results = {'rmse': rmse_train,
                    'mape' : mape_train,
                    'predictions' : y_train_pred}

Evaluating the model with TRAIN data


In [364]:
train_results

{'rmse': 0.6781111999765105,
 'mape': 20.54832458104446,
 'predictions': array([4.1716933, 3.9454303, 3.8573043, ..., 3.7118096, 3.7118096,
        4.1530914], dtype=float32)}

In [289]:
sorted(sklearn.metrics.SCORERS.keys())

['accuracy',
 'adjusted_mutual_info_score',
 'adjusted_rand_score',
 'average_precision',
 'balanced_accuracy',
 'completeness_score',
 'explained_variance',
 'f1',
 'f1_macro',
 'f1_micro',
 'f1_samples',
 'f1_weighted',
 'fowlkes_mallows_score',
 'homogeneity_score',
 'jaccard',
 'jaccard_macro',
 'jaccard_micro',
 'jaccard_samples',
 'jaccard_weighted',
 'max_error',
 'mutual_info_score',
 'neg_brier_score',
 'neg_log_loss',
 'neg_mean_absolute_error',
 'neg_mean_absolute_percentage_error',
 'neg_mean_gamma_deviance',
 'neg_mean_poisson_deviance',
 'neg_mean_squared_error',
 'neg_mean_squared_log_error',
 'neg_median_absolute_error',
 'neg_root_mean_squared_error',
 'normalized_mutual_info_score',
 'precision',
 'precision_macro',
 'precision_micro',
 'precision_samples',
 'precision_weighted',
 'r2',
 'rand_score',
 'recall',
 'recall_macro',
 'recall_micro',
 'recall_samples',
 'recall_weighted',
 'roc_auc',
 'roc_auc_ovo',
 'roc_auc_ovo_weighted',
 'roc_auc_ovr',
 'roc_auc_ovr_we

### optimizer takes 1.5 hours to run, run when have time

In [290]:
from sklearn.model_selection import GridSearchCV
param_grid = dict(
    n_jobs=[16,20,25],
    n_estimators=[100,200,300,400],
    booster = ['gbtree'],
    learning_rate = [0.01,0.02,0.1,1])

xgb_model = xgb.XGBRegressor(silent=False, random_state=15)

grid_search = GridSearchCV(estimator=xgb_model,
                           param_grid=param_grid,
                           scoring='neg_root_mean_squared_error')

best_model = grid_search.fit(x_train, y_train)
print('Optimum parameters', best_model.best_params_)

[15:06:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:07:04] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:07:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:10:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:10:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:10:55] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:17:12] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:17:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:18:03] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:25:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:25:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:25:37] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:29:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:29:45] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:30:00] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

...

[15:44:23] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:44:30] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:44:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:49:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:49:57] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:50:18] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[15:58:52] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:59:27] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[15:59:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[16:03:19] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:33] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:03:47] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

[16:09:38] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:09:59] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[16:10:20] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindi

### Optimum parameters {'booster': 'gbtree', 'learning_rate': 0.1, 'n_estimators': 200, 'n_jobs': 16}
### re-fit the training data 

In [365]:
#re-fit the training data 

# initialize XGBoost model
xgb_model = xgb.XGBRegressor(silent=False, n_jobs=16, random_state=15, n_estimators=200, booster="gbtree",learning_rate= 0.3)
# dictionaries for storing train and test results
train_results = dict()
test_results = dict()
    
    
# fit the model
print('Training the model..')
start =datetime.now()
xgb_model.fit(x_train, y_train, eval_metric = 'rmse')
print('Done. Time taken : {}\n'.format(datetime.now()-start))
print('Done \n')

# from the trained model, get the predictions
print('Evaluating the model with TRAIN data')
start =datetime.now()
y_train_pred = xgb_model.predict(x_train)
# get the rmse and mape of train data...
rmse_train, mape_train = get_error_metrics(y_train.values, y_train_pred)
    
# store the results in train_results dictionary..
train_results = {'rmse': rmse_train,
                    'mape' : mape_train,
                    'predictions' : y_train_pred}

train_results

Training the model..
[18:24:08] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:573: 
Parameters: { "silent" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


Done. Time taken : 0:00:22.491001

Done 

Evaluating the model with TRAIN data


{'rmse': 0.6296320640860958,
 'mape': 18.70491483404222,
 'predictions': array([4.2853975, 4.1073203, 3.8666654, ..., 3.6224732, 3.6139646,
        4.19047  ], dtype=float32)}

MAPE value of 18.7% means that the average difference between the forecasted value and the actual value is 18.7%. It is not the best but the best that the model could get

In [366]:
#######################################
# get the test data predictions and compute rmse and mape
print('Evaluating Test data')
y_test_pred = xgb_model.predict(x_test) 
rmse_test, mape_test = get_error_metrics(y_true=y_test.values, y_pred=y_test_pred)
# store them in our test results dictionary.
test_results = {'rmse': rmse_test,
                    'mape' : mape_test,
                    'predictions':y_test_pred}

Evaluating Test data


In [367]:
test_results

{'rmse': 0.6716533739910432,
 'mape': 19.677372481866758,
 'predictions': array([3.7274673, 3.0524168, 4.2830563, ..., 4.580515 , 4.1454554,
        3.2909656], dtype=float32)}

In [372]:
len(y_train_pred)

80668

In [373]:
len(final_data)

80668

In [374]:
#add new predictions to final_data

final_data['predicted rating'] = y_train_pred
final_data

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd,predicted rating
0,1,1,3.519977,2.0,5.0,4.0,4.0,4.5,3.0,4.0,3.0,5.0,5.0,4.366379,3.954545,4.0,4.306195,4.285398
0,5,1,3.519977,4.0,5.0,4.0,4.0,5.0,4.0,3.0,3.0,5.0,5.0,3.636364,3.954545,4.0,3.941735,4.107320
0,7,1,3.519977,4.0,4.0,5.0,4.5,4.0,4.5,4.5,5.0,4.0,3.0,3.230263,3.954545,4.5,4.543556,3.866665
0,15,1,3.519977,5.0,3.0,4.0,4.0,4.0,3.5,3.0,5.0,3.0,3.0,3.448148,3.954545,2.5,4.677380,3.102271
0,17,1,3.519977,4.0,5.0,4.0,4.0,4.5,4.0,4.5,5.0,5.0,5.0,4.209524,3.954545,4.5,4.810359,4.380750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,184,193581,3.519977,4.0,4.0,4.0,4.0,4.0,5.0,3.5,4.0,3.5,4.5,3.705224,4.000000,4.0,3.090088,4.118227
0,184,193583,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,2.371393,3.625142
0,184,193585,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,3.064229,3.622473
0,184,193587,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,3.254303,3.613965


In [375]:
#add new predictions to final_test_data

final_test_data['predicted rating'] = y_test_pred
final_test_data

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd,predicted rating
0,514,1,3.427881,4.0,4.00,4.00,4.00,4.00,4.0,4.0,4.0,2.5,5.0,3.311083,3.769231,4.0,3.425865,3.727467
0,517,1,3.427881,4.0,4.00,4.00,2.50,4.00,3.5,2.0,3.0,3.5,5.0,2.386250,3.769231,4.0,3.229036,3.052417
0,522,1,3.427881,5.0,4.00,4.00,4.00,3.00,3.5,4.0,5.0,5.0,5.0,3.830000,3.769231,3.0,3.026007,4.283056
0,524,1,3.427881,5.0,4.00,5.00,3.00,4.00,3.0,3.0,3.0,5.0,5.0,3.458015,3.769231,4.0,3.004834,3.441464
0,525,1,3.427881,4.0,3.00,5.00,2.50,4.00,4.0,4.0,4.0,4.5,4.0,3.542000,3.769231,4.0,2.366412,3.921979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,586,187593,3.427881,3.5,3.75,3.75,3.75,3.75,5.0,3.5,3.5,4.0,5.0,4.365385,3.750000,4.0,3.450431,4.384908
0,514,187595,3.427881,5.0,4.00,4.00,4.00,4.00,3.5,2.0,4.0,2.0,3.0,3.311083,4.000000,3.0,3.784835,3.433254
0,586,187595,3.427881,3.0,4.00,4.00,4.00,4.00,4.0,4.5,4.0,4.0,4.0,4.365385,4.000000,5.0,3.561571,4.580515
0,596,188301,3.427881,4.0,4.00,4.00,4.00,4.00,4.0,4.5,2.5,4.0,4.0,3.495134,4.000000,4.0,3.921782,4.145455


In [376]:
complete_df= final_test_data.append(final_data, ignore_index=True)
complete_df

,user,movie,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd,predicted rating
0,514,1,3.427881,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.5,5.0,3.311083,3.769231,4.0,3.425865,3.727467
1,517,1,3.427881,4.0,4.0,4.0,2.5,4.0,3.5,2.0,3.0,3.5,5.0,2.386250,3.769231,4.0,3.229036,3.052417
2,522,1,3.427881,5.0,4.0,4.0,4.0,3.0,3.5,4.0,5.0,5.0,5.0,3.830000,3.769231,3.0,3.026007,4.283056
3,524,1,3.427881,5.0,4.0,5.0,3.0,4.0,3.0,3.0,3.0,5.0,5.0,3.458015,3.769231,4.0,3.004834,3.441464
4,525,1,3.427881,4.0,3.0,5.0,2.5,4.0,4.0,4.0,4.0,4.5,4.0,3.542000,3.769231,4.0,2.366412,3.921979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
100831,184,193581,3.519977,4.0,4.0,4.0,4.0,4.0,5.0,3.5,4.0,3.5,4.5,3.705224,4.000000,4.0,3.090088,4.118227
100832,184,193583,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,2.371393,3.625142
100833,184,193585,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,3.064229,3.622473
100834,184,193587,3.519977,3.5,3.5,3.5,3.5,3.5,5.0,3.5,4.0,3.5,4.5,3.705224,3.500000,3.5,3.254303,3.613965


In [377]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [390]:
complete_df=complete_df.rename(columns={"movieID": "movieId"})
complete_df = pd.merge(movies, complete_df)
complete_df.head()

,movieId,title,genres,user,GAvg,sur1,sur2,sur3,sur4,sur5,smr1,smr2,smr3,smr4,smr5,UAvg,MAvg,rating,mf_svd,predicted rating
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,514,3.427881,4.0,4.0,4.0,4.0,4.0,4.0,4.0,4.0,2.5,5.0,3.311083,3.769231,4.0,3.425865,3.727467
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,517,3.427881,4.0,4.0,4.0,2.5,4.0,3.5,2.0,3.0,3.5,5.0,2.386250,3.769231,4.0,3.229036,3.052417
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,522,3.427881,5.0,4.0,4.0,4.0,3.0,3.5,4.0,5.0,5.0,5.0,3.830000,3.769231,3.0,3.026007,4.283056
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,524,3.427881,5.0,4.0,5.0,3.0,4.0,3.0,3.0,3.0,5.0,5.0,3.458015,3.769231,4.0,3.004834,3.441464
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,525,3.427881,4.0,3.0,5.0,2.5,4.0,4.0,4.0,4.0,4.5,4.0,3.542000,3.769231,4.0,2.366412,3.921979


In [448]:
#Export individual file 
output_data_file = "../cleaned data/complete_df_with_predictions.csv"
complete_df.to_csv(output_data_file, index = False)